## Segmenting and Clustering Neighborhoods in Toronto

### STEP 1 - Scrape the Toronto Wikipedia page and wrangle the data

In [1]:
#conda install -c anaconda Beautifulsoup4
# Done

In [2]:
# urllib.request is a Python module for fetching URLs (Uniform Resource Locators)
import urllib.request
# Beautiful Soup is a Python library for pulling data out of HTML and XML files
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np

In [3]:
# Fetch the html file
response = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
html_doc = response.read()

# Parse the html file
soup = BeautifulSoup(html_doc, 'html.parser')

# Format the parsed html file
strhtm = soup.prettify()

# Print the first few characters
print (strhtm[:225])

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-j


In [4]:
column_names=['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns=column_names)

In [5]:
df.head()

,Postalcode,Borough,Neighborhood


In [6]:
postalcode_tb=soup.find('table')

In [7]:
## Extracting information from the table
# Match row
for trmark in postalcode_tb.find_all('tr'):
    data_list=[]
    # Match field
    for tdmark in trmark.find_all('td'):
        # Append to list
        data_list.append(tdmark.text.strip())
    if len(data_list)==3:
        # Append to dataframe
        df.loc[len(df)] = data_list

####  The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [8]:
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [9]:
df.shape

(180, 3)

#### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned

In [10]:
#Delete rows where Borough is 'Not assigned'
df = df[df.Borough!= "Not assigned"]

In [11]:
df.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [12]:
df

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


#### If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.

In [13]:
# Neighborhood=Borough when Neighborhood is 'Not assigned'
df['Neighborhood'] = df['Borough'].where(df['Neighborhood'] == 'Not assigned', df['Neighborhood'])


In [14]:
# Just to Check - df.where(df['Neighborhood'] == "Not assigned")

In [15]:
df.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#### More than one neighborhood can exist in one postal code area. The rows will be combined into one row with the neighborhoods separated.

In [16]:
# Concatenate all neighbourhoods where postcode and Borough are the same
df = df.groupby(['Postalcode', 'Borough'])['Neighborhood'].apply(lambda x: ', '.join(x))

In [17]:
df.reset_index()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [18]:
df_temp = pd.DataFrame(df)
df_final = df_temp.reset_index()
df_final.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [19]:
df_final.shape

(103, 3)